In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import time

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from transformers import AutoTokenizer, AutoModel
from transformers import LlamaForCausalLM
import torch

TOKENIZER_PATH =  '/home/lishengping/mengqy/projects/slim_alignment/tokenizer'
qwen_tokenizer = AutoTokenizer.from_pretrained(
            TOKENIZER_PATH, use_fast=False, trust_remote_code=True
        )


SPECIAL_TOKENS: ((151643, '<|endoftext|>'), (151644, '<|im_start|>'), (151645, '<|im_end|>'), (151646, '<|extra_0|>'), (151647, '<|extra_1|>'), (151648, '<|extra_2|>'), (151649, '<|extra_3|>'), (151650, '<|extra_4|>'), (151651, '<|extra_5|>'), (151652, '<|extra_6|>'), (151653, '<|extra_7|>'), (151654, '<|extra_8|>'), (151655, '<|extra_9|>'), (151656, '<|extra_10|>'), (151657, '<|extra_11|>'), (151658, '<|extra_12|>'), (151659, '<|extra_13|>'), (151660, '<|extra_14|>'), (151661, '<|extra_15|>'), (151662, '<|extra_16|>'), (151663, '<|extra_17|>'), (151664, '<|extra_18|>'), (151665, '<|extra_19|>'), (151666, '<|extra_20|>'), (151667, '<|extra_21|>'), (151668, '<|extra_22|>'), (151669, '<|extra_23|>'), (151670, '<|extra_24|>'), (151671, '<|extra_25|>'), (151672, '<|extra_26|>'), (151673, '<|extra_27|>'), (151674, '<|extra_28|>'), (151675, '<|extra_29|>'), (151676, '<|extra_30|>'), (151677, '<|extra_31|>'), (151678, '<|extra_32|>'), (151679, '<|extra_33|>'), (151680, '<|extra_34|>'), (15168

In [2]:
model_dir = '/home/lishengping/mengqy/data/llama_model/llama-7b-hf-custom'
device = 'cuda:1'
llama_model = LlamaForCausalLM.from_pretrained(model_dir, device_map='cpu')
model_dir = '/home/lishengping/mengqy/data/llama_model/llama-7b-hf-custom'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
llama_model = llama_model.half()
llama_model = llama_model.to(device)
llama_model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [3]:
import os
import time
import argparse
import socket
import random
from collections import defaultdict
os.environ["JAX_PLATFORMS"] = "cpu"

import tensorflow as tf
import jax
import numpy as np


seq_len = 32769

def _parse_function(example_proto):
    # seq_len = 1024
    feature_desc = {key: tf.io.VarLenFeature(tf.int64) for key in task_features}
    example = tf.io.parse_single_example(example_proto, feature_desc)
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.cast(t, dtype=tf.int32)
        # example[name] = tf.sparse.to_dense(t, default_value=0)[: 2 * seq_len - 2]
        # example[name] = tf.reshape(example[name], [2, seq_len - 1])
        # t = tf.constant([[10], [10]], dtype=tf.int32)
        # example[name] = tf.concat([t, example[name]], 1)
        example[name] = tf.sparse.to_dense(t, default_value=0)[: seq_len]
        print(f'example[name]: {example[name]}')
    return example

task_features = {'input_ids': None}
train_seed = 1234
num_infeed_hosts = 1
shuffle_buffer_size = None
pad_id = 0
batch_size = 1

fname = ['gs://jax_llm_data_us-east5/xiaomeng/v3.5/tfids_4k_32k_0622/B009/F009/000.long']
# fname = ['gs://jax_llm_data_us-east5/xiaomeng/v3.5/tfids_4k_32k_0622/valid_tfrecord/001.long']
# fname = ['gs://jax_llm_data/xiaomeng/sft_target/tfrecord_len2k/en.test.continue_write.tfrecord']
tf.random.set_seed(train_seed)
ds = tf.data.Dataset.from_tensor_slices(fname)
ds = ds.apply(tf.data.TFRecordDataset)
# shard host data
# ds = ds.shard(num_infeed_hosts, 0)
ds = ds.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
if shuffle_buffer_size is not None:
    ds = ds.shuffle(buffer_size=self.shuffle_buffer_size)
padded_shapes = {key: seq_len for key in task_features}
padding_values = {key: pad_id for key in task_features}
ds = ds.padded_batch(
    batch_size=np.prod(batch_size),
    padded_shapes=padded_shapes,
    padding_values=padding_values,
    drop_remainder=True,
)
# ds = ds.map(self.convert)
# ds = ds.prefetch(tf.data.AUTOTUNE)
iter_ds = ds.as_numpy_iterator()

2024-07-19 09:23:09.386769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib


example[name]: Tensor("strided_slice:0", shape=(None,), dtype=int32)


2024-07-19 09:23:11.177845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2024-07-19 09:23:11.177950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2024-07-19 09:23:11.178016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib6

In [5]:
# text = ''''In a world brimming with heartwarming stories, few are as touching and inspiring as those of adoption. These tales are a testament to the extraordinary power of love, compassion, and the unbreakable bonds that form when individuals open their hearts to welcome a child into their lives. Join us on this heartwarming journey as we explore how adoption transforms strangers into loving families, and let the hope and inspiration they bring warm your heart. 
# A Journey of Love

# Adoption is more than a legal process; it's a profound journey of love. It's the story of parents, often complete strangers to a child, who make a conscious choice to love, nurture, and provide a forever home to a little one in need. It's a commitment that transcends biology, built upon the foundation of selflessness and boundless love. 


# 1.The Miracle of Matching

# The process of adoption can be a complex one, involving home studies, interviews, and legal procedures. However, amid the paperwork and formalities, there's a remarkable moment of magic when a child is matched with their forever family. It's a moment filled with hope and anticipation, the beginning of a new chapter in the lives of both child and parent. 


# The day of adoption finalization is a joyous occasion, marking the official start of a lifelong journey together. The child finds their forever family, and the parents find their missing piece. From that day forward, an unbreakable bond is forged, one that grows stronger with each passing day, filled with love, laughter, and shared experiences. '''
start = time.time()
N = 10
with torch.no_grad():
    for i in range(N):
        long_sample = next(iter_ds)
        qwen_input_ids = long_sample['input_ids']
        qwen_text = qwen_tokenizer.decode(qwen_input_ids[0, :8000])
        text = qwen_text
        input_ids = tokenizer.encode(text, return_tensors='pt')
        if input_ids.device.type == 'cpu':
            input_ids = input_ids.to(device)
        result = llama_model(input_ids[:, :], labels=input_ids[:, :])
        save_dict = {'text': text, 'input_ids': input_ids, 'loss': result.loss.cpu()}
        pickle.dump(save_dict, open('llama2-7b.long.eval.1k.pkl', 'ab'))
        print(f'Processing: {i}/1000, take: {time.time() - start:.3f}s')
        del result
        del input_ids
        del save_dict

Processing: 0/1000, take: 2.797s
Processing: 1/1000, take: 3.599s
Processing: 2/1000, take: 4.400s
Processing: 3/1000, take: 5.214s
Processing: 4/1000, take: 6.017s
Processing: 5/1000, take: 7.678s
Processing: 6/1000, take: 9.319s
Processing: 7/1000, take: 10.875s
Processing: 8/1000, take: 12.489s
Processing: 9/1000, take: 13.174s
